In [1]:
import numpy as np
import pandas as pd
import warnings
from tqdm import TqdmExperimentalWarning

warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from traffic.core import Traffic
from datetime import timedelta

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

In [2]:
t = Traffic.from_file("../data/2022-01-01.parquet")
t

,count
flight_id,
248763716,29635
248763259,27916
248752988,27683
248762188,26719
248757757,26505
248751599,26222
248765179,26217
248766045,26147
248757924,25816


In [3]:
t.data

,flight_id,timestamp,latitude,longitude,altitude,groundspeed,track,vertical_rate,icao24,u_component_of_wind,v_component_of_wind,temperature,specific_humidity
0,248763775,2022-01-01 18:14:38+00:00,30.898407,31.240270,38000.0,429.0,350.888643,-64.0,248763775,8.754636,-13.070078,221.819618,0.000008
1,248763775,2022-01-01 18:14:39+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.696749,-13.106524,221.794782,0.000008
2,248763775,2022-01-01 18:14:40+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697087,-13.107184,221.794700,0.000008
3,248763775,2022-01-01 18:14:41+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697424,-13.107845,221.794618,0.000008
4,248763775,2022-01-01 18:14:42+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697762,-13.108505,221.794536,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11862458,248772010,2022-01-02 01:59:55+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665934,-31.158741,210.208665,0.000013
11862459,248772010,2022-01-02 01:59:56+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665919,-31.158539,210.208654,0.000013
11862460,248772010,2022-01-02 01:59:57+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665905,-31.158336,210.208643,0.000013
11862461,248772010,2022-01-02 01:59:58+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665890,-31.158134,210.208632,0.000013


# encoding

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [5]:
def encode_datetime(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [6]:
t.data['timestamp'] = pd.to_datetime(t.data['timestamp'], utc=True)
t.data['timestamp_month'] = t.data["timestamp"].dt.month
t.data['timestamp_day'] = t.data["timestamp"].dt.day
t.data['timestamp_hour'] = t.data["timestamp"].dt.hour
t.data['timestamp_minue'] = t.data["timestamp"].dt.minute
encode_datetime(t.data, 'timestamp_month', 12)
encode_datetime(t.data, 'timestamp_day', 31)
encode_datetime(t.data, 'timestamp_hour', 24)
encode_datetime(t.data, 'timestamp_minue', 60)

,flight_id,timestamp,latitude,longitude,altitude,groundspeed,track,vertical_rate,icao24,u_component_of_wind,...,timestamp_hour,timestamp_minue,timestamp_month_sin,timestamp_month_cos,timestamp_day_sin,timestamp_day_cos,timestamp_hour_sin,timestamp_hour_cos,timestamp_minue_sin,timestamp_minue_cos
0,248763775,2022-01-01 18:14:38+00:00,30.898407,31.240270,38000.0,429.0,350.888643,-64.0,248763775,8.754636,...,18,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528
1,248763775,2022-01-01 18:14:39+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.696749,...,18,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528
2,248763775,2022-01-01 18:14:40+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697087,...,18,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528
3,248763775,2022-01-01 18:14:41+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697424,...,18,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528
4,248763775,2022-01-01 18:14:42+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697762,...,18,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11862458,248772010,2022-01-02 01:59:55+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665934,...,1,59,0.5,0.866025,0.394356,0.918958,0.258819,9.659258e-01,-0.104528,0.994522
11862459,248772010,2022-01-02 01:59:56+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665919,...,1,59,0.5,0.866025,0.394356,0.918958,0.258819,9.659258e-01,-0.104528,0.994522
11862460,248772010,2022-01-02 01:59:57+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665905,...,1,59,0.5,0.866025,0.394356,0.918958,0.258819,9.659258e-01,-0.104528,0.994522
11862461,248772010,2022-01-02 01:59:58+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665890,...,1,59,0.5,0.866025,0.394356,0.918958,0.258819,9.659258e-01,-0.104528,0.994522


In [7]:
le.fit(t.data['icao24'])
t.data['icao24_category'] = le.transform(t.data['icao24']) + 1
t.data[['icao24','icao24_category']]

,icao24,icao24_category
0,248763775,1447
1,248763775,1447
2,248763775,1447
3,248763775,1447
4,248763775,1447
...,...,...
11862458,248772010,1708
11862459,248772010,1708
11862460,248772010,1708
11862461,248772010,1708


In [8]:
t.data

,flight_id,timestamp,latitude,longitude,altitude,groundspeed,track,vertical_rate,icao24,u_component_of_wind,...,timestamp_minue,timestamp_month_sin,timestamp_month_cos,timestamp_day_sin,timestamp_day_cos,timestamp_hour_sin,timestamp_hour_cos,timestamp_minue_sin,timestamp_minue_cos,icao24_category
0,248763775,2022-01-01 18:14:38+00:00,30.898407,31.240270,38000.0,429.0,350.888643,-64.0,248763775,8.754636,...,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528,1447
1,248763775,2022-01-01 18:14:39+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.696749,...,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528,1447
2,248763775,2022-01-01 18:14:40+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697087,...,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528,1447
3,248763775,2022-01-01 18:14:41+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697424,...,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528,1447
4,248763775,2022-01-01 18:14:42+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697762,...,14,0.5,0.866025,0.201299,0.979530,-1.000000,-1.836970e-16,0.994522,0.104528,1447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11862458,248772010,2022-01-02 01:59:55+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665934,...,59,0.5,0.866025,0.394356,0.918958,0.258819,9.659258e-01,-0.104528,0.994522,1708
11862459,248772010,2022-01-02 01:59:56+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665919,...,59,0.5,0.866025,0.394356,0.918958,0.258819,9.659258e-01,-0.104528,0.994522,1708
11862460,248772010,2022-01-02 01:59:57+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665905,...,59,0.5,0.866025,0.394356,0.918958,0.258819,9.659258e-01,-0.104528,0.994522,1708
11862461,248772010,2022-01-02 01:59:58+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665890,...,59,0.5,0.866025,0.394356,0.918958,0.258819,9.659258e-01,-0.104528,0.994522,1708
